## Batching the data

In [ ]:
batch_size = 1000

X_loader = DataLoader(X, batch_size=batch_size, shuffle=False)
Y_loader = DataLoader(Y, batch_size=batch_size, shuffle=False)

total_batches = len(X_loader)
print(total_batches)

## Current Loss

In [ ]:
min_loss = 1

# seperating the Y
Y1 = Y[:, :8]
Y2 = Y[:, 8:16]
Y3 = Y[:, 16:24]
Y4 = Y[:, 24:32]
Y5 = Y[:, 32:]

Y_list = [Y1,Y2,Y3,Y4,Y5]

with torch.no_grad():
    y_preds = model(X.to(device)) # will output a tuple of 5 tensors

    total_loss = 0
    for i in range(len(y_preds)): # calculating the loss per tensor
        y_pred = y_preds[i]
        total_loss += criterion(y_pred, Y_list[i].to(device))

    print('Current loss:', f'{total_loss.item()*100:.3f}%')
    min_loss = total_loss


## Training

### No batching

In [ ]:
# seperating the Y
Y1 = Y[:, :8]
Y2 = Y[:, 8:16]
Y3 = Y[:, 16:24]
Y4 = Y[:, 24:32]
Y5 = Y[:, 32:]

Y_list = [Y1,Y2,Y3,Y4,Y5]

for epoch in range(n_epochs):
    # forward
    y_preds = model(X.to(device)) # will output a tuple of 5 tensors

    total_loss = 0
    for i in range(len(y_preds)): # calculating the loss per tensor
        y_pred = y_preds[i]
        total_loss += criterion(y_pred, Y_list[i].to(device))

    # backward
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if (epoch) % 5 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {total_loss.item():.4f}', end='\r')
    
    if total_loss.item() < min_loss:
        min_loss = total_loss.item()
        print('model saved, loss:', min_loss)
        torch.save(model.state_dict(), './models/'+str(model._get_name())+'_'+str(elo_or_year)+'.pt')

### Batching

In [ ]:
for epoch in range(n_epochs):
    batch = 0
    for X_batch, Y_batch in zip(X_loader, Y_loader):
        X_batch = X_batch.to(device)
        Y_batch = Y_batch.to(device)
        # forward
        y_preds = model(X_batch) # will output a tuple of 5 tensors

        # seperating the Y
        Y1 = Y_batch[:, :8]
        Y2 = Y_batch[:, 8:16]
        Y3 = Y_batch[:, 16:24]
        Y4 = Y_batch[:, 24:32]
        Y5 = Y_batch[:, 32:]

        Y_list = [Y1,Y2,Y3,Y4,Y5]

        total_loss = 0
        for i in range(len(y_preds)): # calculating the loss per tensor
            y_pred = y_preds[i]
            total_loss += criterion(y_pred, Y_list[i])

        # backward
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        if (batch+1) % 3 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], Batch [{batch+1}/{total_batches}], Loss: {total_loss.item():.4f}', end='\r')
        batch+=1